RESNET, nfolds=5, 50-15-15, patience=3, resize=128, DataAugment= zoom 0.2 shear_range0.2 , optimizers.Adam, activation=sigmoid

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from subprocess import check_output
#print(check_output(["ls", ".."]).decode("utf8"))

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
from tqdm import tqdm

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

from sklearn.cross_validation import KFold
from sklearn.metrics import fbeta_score
import time

In [ ]:
x_train = []
x_test = []
y_train = []

df_train = pd.read_csv('train_v2.csv')
df_test = pd.read_csv('sample_submission_v2.csv')

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

In [ ]:
labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'cultivation',
 'artisinal_mine',
 'haze',
 'primary',
 'slash_burn',
 'habitation',
 'clear',
 'road',
 'selective_logging',
 'partly_cloudy',
 'agriculture',
 'water',
 'cloudy']

In [ ]:
label_map = {'agriculture': 14,
 'artisinal_mine': 5,
 'bare_ground': 1,
 'blooming': 3,
 'blow_down': 0,
 'clear': 10,
 'cloudy': 16,
 'conventional_mine': 2,
 'cultivation': 4,
 'habitation': 9,
 'haze': 6,
 'partly_cloudy': 13,
 'primary': 7,
 'road': 11,
 'selective_logging': 12,
 'slash_burn': 8,
 'water': 15}

In [ ]:
for f, tags in tqdm(df_train.values, miniters=1000):
    img = cv2.imread('./train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(img, (128, 128)))
    y_train.append(targets)

In [ ]:
x_train = np.array(x_train, np.float32)/255.

In [ ]:
y_train = np.array(y_train, np.uint8)

In [ ]:
for f, tags in tqdm(df_test.values, miniters=1000):
    img = cv2.imread('./test-jpg/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (128, 128)))

In [ ]:
x_test  = np.array(x_test, np.float32)/255.

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
nfolds = 5

num_fold = 0
sum_score = 0

yfull_test = []
yfull_train =[]

In [ ]:
batch_size = 128

In [ ]:
kf = KFold(len(y_train), n_folds=nfolds, shuffle=True, random_state=1)

In [ ]:
datagen = ImageDataGenerator(zoom_range=0.2, shear_range=0.2, horizontal_flip=True, vertical_flip=True)

In [ ]:
for train_index, test_index in kf:
        start_time_model_fitting = time.time()
        
        X_train = x_train[train_index]
        Y_train = y_train[train_index]
        X_valid = x_train[test_index]
        Y_valid = y_train[test_index]

        num_fold += 1
        print('Start KFold number {} from {}'.format(num_fold, nfolds))
        print('Split train: ', len(X_train), len(Y_train))
        print('Split valid: ', len(X_valid), len(Y_valid))
        
        kfold_weights_path = os.path.join('', 'weights_kfold_' + str(num_fold) + '.h5')
        
        model = Sequential()
        model.add(BatchNormalization(input_shape=(128, 128, 3)))
        model.add(Conv2D(32, kernel_size=(3, 3),padding='same', activation='relu'))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256, kernel_size=(3, 3),padding='same', activation='relu'))
        model.add(Conv2D(256, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
             
        model.add(Conv2D(512, kernel_size=(3, 3),padding='same', activation='relu'))
        model.add(Conv2D(512, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, kernel_size=(3, 3),padding='same', activation='relu'))
        model.add(Conv2D(1024, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        model.add(Flatten())
        model.add(Dense(1024, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(17, activation='sigmoid'))

        epochs_arr = [50, 15, 15]
        learn_rates = [0.001, 0.0001, 0.00001]

        for learn_rate, epochs in zip(learn_rates, epochs_arr):
            opt  = optimizers.Adam(lr=learn_rate)
            model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
                          optimizer=opt,
                          metrics=['accuracy'])
            callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=0),
            ModelCheckpoint(kfold_weights_path, monitor='val_loss', save_best_only=True, verbose=0)]

            #model.fit(x = X_train, y= Y_train, validation_data=(X_valid, Y_valid),
                  #batch_size=128,verbose=2, epochs=epochs,callbacks=callbacks,shuffle=True)
            
            model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                            validation_data=datagen.flow(X_valid, Y_valid, batch_size=batch_size),
                            validation_steps=len(X_valid) / batch_size,
                            steps_per_epoch=len(X_train) / batch_size,
                            epochs=epochs,
                            callbacks=callbacks,
                            workers=4)
        
        if os.path.isfile(kfold_weights_path):
            model.load_weights(kfold_weights_path)
        
        p_valid = model.predict(X_valid, batch_size = batch_size, verbose=2)
        print(fbeta_score(Y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))

        p_train = model.predict(x_train, batch_size = batch_size, verbose=2)
        yfull_train.append(p_train)
        
        p_test = model.predict(x_test, batch_size = batch_size, verbose=2)
        yfull_test.append(p_test)

In [ ]:
result = np.array(yfull_test[0])
for i in range(1, nfolds):
    result += np.array(yfull_test[i])
result /= nfolds
result = pd.DataFrame(result, columns = labels)
result

In [ ]:
from tqdm import tqdm
thres = [0.07, 0.17, 0.2, 0.04, 0.23, 0.33, 0.24, 0.22, 0.1, 0.19, 0.23, 0.24, 0.12, 0.14, 0.25, 0.26, 0.16]
preds = []
for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.ix[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

In [ ]:
p_valid = model.predict(X_valid, batch_size = 128, verbose=2)
print(fbeta_score(Y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))

In [ ]:
# Let's save those weights.
model.save_weights('Kele_XU_Keras_0913-Copy9-ter-ResNet50-96.h5')

In [ ]:
df_test['tags'] = preds
df_test.to_csv('submission_keras_5_fold_CV_0.9136_LB_0.913_copy9-ter-RestNet50-96.csv', index=False)